In [1]:
import cv2
import glob
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

## ImageAugmentation

In [16]:
class ImageAugmentation:
    def __init__(self, path):
        self.dim = (1280, 720)
        self.folder_list = glob.glob(path + "/*")
        self.folders = []
        for p in self.folder_list:
            self.folders.append(p.split("\\")[-1])
        
        
        for folder in self.folders:
            self.file_names = []
            self.files = glob.glob(path + "/" + folder + "/*")
            for file in self.files:
                #resize
                self.file_names.append(file.split("\\")[-1])
                img = cv2.imread(path + "/" + folder + "/" + self.file_names[-1])
                resized = cv2.resize(img, self.dim,interpolation=cv2.INTER_AREA)                
                cv2.imwrite(path + "/" + folder + "/" + self.file_names[-1],resized)
                
                #resized img reflection
                if self.file_names[-1][:2] != "r_":
                    rows, cols, dim = resized.shape
                    M = np.float32([[-1,  0, cols],
                                    [0 ,  1, 0   ],
                                    [0 ,  0, 1   ]])
                    reflected_img = cv2.warpPerspective(resized,M,(int(cols),int(rows)))
                    cv2.imwrite(path + "/" + folder + "/" + "r_"+ self.file_names[-1],reflected_img)
                
            
                
        

In [15]:
ImageAugmentation("./data")

UnboundLocalError: local variable 'resized' referenced before assignment

In [63]:
class CDataset(Dataset):
    def __init__(self):
        self.data_path = "./data/"
        self.folder_list = glob.glob(self.data_path + "*")

        self.data = []
        for folder in self.folder_list:
            folder_name = folder.split("\\")[-1]
            for img_path in glob.glob(self.data_path + folder_name + "/*.jpg"):
                self.data.append([img_path, folder_name])
        self.class_map = {"Car": 0, "Truck": 1}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        img_path, class_name = self.data[i]
        img = cv2.imread(img_path)
        class_id = self.class_map[class_name]
        img_tensor = torch.tensor(img, dtype=torch.float)
        img_tensor = img_tensor.permute(2,0,1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id

In [65]:
if __name__ == "__main__":
    dataset = CDataset()
    data_loader = DataLoader(dataset, batch_size=5, shuffle=True)
    
    for imgs, labels in data_loader:
        print("Batch of images has shape: ",imgs.shape)
        print("Batch of labels has shape: ", labels.shape)

Batch of images has shape:  torch.Size([5, 3, 720, 1280])
Batch of labels has shape:  torch.Size([5, 1])
Batch of images has shape:  torch.Size([5, 3, 720, 1280])
Batch of labels has shape:  torch.Size([5, 1])
